# Load cleaned CSV files, normalize merchant names, and create monthly buckets to detect subscription services

In [ ]:
import pandas as pd
import numpy as np

current_statement = "../data/checking-account-aggregated-cleaned.csv"

# Load cleaned CSV with correct data types, sort by index

In [ ]:
df = pd.read_csv(
    current_statement,
    parse_dates=["Transaction Date", "Posted Date"],
    dtype={
        "Transaction Type": "string",
        "Check/Serial #": "float64",
        "Full description": "string",
        "Merchant name": "string",
        "Category name": "string",
        "Sub-category name": "string",
        "Amount": "float64",
        "Daily Posted Balance": "string"
    },
    index_col="Transaction Date",
)

df = df.sort_index()

# Normalize merchant names

In [ ]:
df["merchant_norm"] = (
    df["Merchant name"]
    .str.upper()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.replace(r"\d{2,}", "", regex=True) # strip long number sequences
    .str.replace(r"(POS|ACH|DEBIT|CREDIT|RECURRING)", "", regex=True)
    .str.strip()
)
df.head(10)

# Keep only outgoing charges (amount < 0)

In [ ]:
df = df[df["Amount"] < 0].copy()
df["amount_abs"] = df["Amount"].abs()

# Create monthly buckets via Grouper and aggregate by merchant and month

In [ ]:
monthly = (
    df
    .groupby([
        "merchant_norm",
        pd.Grouper(freq="ME")
    ])
    .agg(
        tx_count=("amount_abs", "size"),
        amount_mean=("amount_abs", "mean")
    )
    .reset_index()
)
monthly.head(100)

# Aggregate per-merchant statistics across months

In [ ]:
merchant_stats = (
    monthly
    .groupby("merchant_norm")
    .agg(
        months_active=("Transaction Date", "nunique"),
        avg_amount=("amount_mean", "mean"),
        amount_std=("amount_mean", "std"),
        avg_tx_per_month=("tx_count", "mean"),
    )
    .reset_index()
)
merchant_stats.head(100)

# Apply subscription heuristics:

at least 3 months active, average amount stddev <= avg_amount * 0.1, average transactions per month <= 1

In [ ]:
subscriptions = merchant_stats[
    (merchant_stats["months_active"] >= 2) &
    (merchant_stats["avg_tx_per_month"] <= 2) &
    (
        merchant_stats["amount_std"].isna() |
        (merchant_stats["amount_std"] <= merchant_stats["avg_amount"] * 0.1)
    )
].copy()
subscriptions.head(500)

# Example charges for explainability

In [ ]:
examples = (
    df
    .merge(subscriptions[["merchant_norm"]], on="merchant_norm")
    .sort_index()
    .groupby("merchant_norm")
    .head(50)
    [["merchant_norm", "amount_abs"]]
)

In [ ]:
subscriptions.sort_values(
    ["months_active", "avg_amount"],
    ascending=[False, False]
)
